## 範例 : (Kaggle)房價預測

以下用房價預測資料, 觀察特徵的幾種類型

這份資料有 'int64', 'float64', 'object' 三種欄位, 分別將其以python的list格式紀錄下來

## [教學目標]

以下程式碼將示範 : 如何將欄位名稱, 依照所屬類型分開, 並列出指定類型的部分資料

## [範例重點]

如何觀察目前的 DataFrame 中, 有哪些欄位類型, 以及數量各有多少 (In[3], Out[3])

如何將欄位名稱依欄位類型分開 (In[4], Out[4])

如何只顯示特定類型的欄位資料 (In[5], Out[5])

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = '/Users/johnsonhuang/py_ds/ML_100/2nd-ML100Days/homework/data/Part02/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')
df_train.shape

(1460, 81)

In [2]:
# 訓練資料需要 train_X, train_Y / 預測輸出需要 ids(識別每個預測值), test_X
# 在此先抽離出 train_Y 與 ids, 而先將 train_X, test_X 該有的資料合併成 df, 先作特徵工程
train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
# 秀出資料欄位的類型, 與對應的數量
# df.dtypes : 轉成以欄位為 index, 類別(type)為 value 的 DataFrame
# .reset_index() : 預設是將原本的 index 轉成一個新的欄位, 如果不須保留 index, 則通常會寫成 .reset_index(drop=True)
dtype_df = df.dtypes.reset_index() 
dtype_df.columns = ["Count", "Column Type"]
dtype_df

,Count,Column Type
0,MSSubClass,int64
1,MSZoning,object
2,LotFrontage,float64
3,LotArea,int64
4,Street,object
5,Alley,object
6,LotShape,object
7,LandContour,object
8,Utilities,object
9,LotConfig,object


In [4]:
grouped_dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
grouped_dtype_df

,Column Type,Count
0,int64,25
1,float64,11
2,object,43


In [5]:
# 確定只有 int64, float64, object 三種類型後對欄位名稱執行迴圈, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
# .dtypes(欄位類型), .columns(欄位名稱) 是 DataFrame 提供的兩個方法, 這裡順便展示一下 for 與 zip 搭配的用法
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
# 這邊採用的寫法稱為 f-string, 是 Python 3.6.2 以後版本才出現的
# 如果無法執行, 則需要更新到這個版本之後, 或自行將程式改寫為 str.format 形式
# 改寫方式可以參考 https://blog.louie.lu/2017/08/08/outdate-python-string-format-and-fstring/
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

25 Integer Features : ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

11 Float Features : ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

43 Object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Fi

In [6]:
#這樣就可以單獨秀出特定類型的欄位集合, 方便做後續的特徵工程處理
df[float_features].head()

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0
1,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0
2,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0
3,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0
4,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0


## 作業 : (Kaggle)鐵達尼生存預測

## [作業目標]

試著完成三種不同特徵類型的三種資料操作, 觀察結果

思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

## [作業重點]

完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])

思考何種特徵類型, 應該最複雜

In [7]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = '/Users/johnsonhuang/py_ds/ML_100/2nd-ML100Days/homework/data/Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [8]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [10]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


## 作業1

試著執行作業程式，觀察三種類型的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique ) 中的九次操作會有那些問題?

並試著解釋那些發生Error的程式區塊的原因?

> error : object型態 的變數無法取 平均 mean / 最大值 Max / 相異值 nunique ，最大最小值也只是字母的排序

> 對 Pclass 這種在意義上為類別變數的欄位取平均及最大最小值沒有意義

In [11]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作
"""
Your Code Here
"""
print("\n --- int ---")
display(df[int_features].mean())
display(df[int_features].max())
display(df[int_features].nunique()) 

print("\n --- float ---")
display(df[float_features].mean())
display(df[float_features].max())
display(df[float_features].nunique()) 

print("\n --- object ---")
display(df[object_features].mean())
display(df[object_features].max())
display(df[object_features].nunique()) 


 --- int ---


Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

Pclass    3
SibSp     8
Parch     9
dtype: int64

Pclass    3
SibSp     7
Parch     8
dtype: int64


 --- float ---


...

Age      98
Fare    281
dtype: int64


 --- object ---


Series([], dtype: float64)

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64

## 作業2

思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?

-------

### 今天五種類型 : 

1. 類別型

> 類別型特徵 : 通常⼀種類別對應⼀種分數

2. 數值型

> 數值型特徵 : 有不同轉換⽅式，函數 / 條件式都可以

3. 二元特徵 : 

> 只有 True / False 兩種數值的特徵；可以當作類別型，也可當作數值型特徵 ( True:1 / False: 0 )

4. 排序型特徵

> 例如名次 / 百分等級，有⼤⼩關係，但並非連續數字；通常當作數值型特徵處理，因為當作類別型會失去排序資訊

5. 時間型特徵：具「週期性」

> 雖然時間型特徵可當作數值型特徵或類別型特徵，但都不適合 ； 取總秒數雖可變為數值，但會失去週期性 (ex ⽉ / 星期) ； 使⽤本⾝可以當作類別，但會失去排序資訊，類別數量也過⼤，因此時間型特徵我們會個別於 Day 25 的課程中講解

-------

### 其他類型 ：

- 文字型特徵
- 可歸類在類別型特徵（object）

> 須將文字經過斷詞、向量化處理後才能做後續的整理




所以三大類特徵中，哪一大類處理起來應該最複雜?

> 類別型特徵

## 補充資料

## [Python Tutorial 第二堂 - 數值與字串型態](https://openhome.cc/Gossip/CodeData/PythonTutorial/NumericStringPy3.html)

焦點主要在於：搭配一些 python 語法，向同學更清楚的展示型態的差異，如果同學看完今天的課程人不是很了解，建議可以搭配此內容理解，當然如果覺得已經能自己分辨，就可以跳過這份參考資料了。

![image](https://lh4.googleusercontent.com/6uJTSMlZSxPFkwVgIHMkCxOaNVrb6NvT4PdJ0HTrGsoL7Ym8EKoxalBq-1L3EqXL_9Pj7wGCG3sw_wceWFg2w4vLVSG9yLD2qeMwC8ZW1GfG25gH5ZLhlVN2vrMZcwgljsO3fk9FFT8?t=1556780162425)

## [Python3.7.2 : Built-in Types Python 官方說明](https://docs.python.org/3/library/stdtypes.html)

這份官方網頁內容資料則是從語言的角度，列舉這些型態，以及對應的操作，因為是官方網頁，會隨著版本更新而略有修改，建議同學有需要的時候(例如版本變更)，再來查詢即可，不用細讀。

![image](https://ai100-fileentity.cupoy.com/2nd/homework/D18/1556779757813/large)